# Projet Régression Linéaire


Mini-projet : Exploration de la Grid Search et de la Validation Croisée
Ce projet avait pour but de me familiariser avec deux techniques fondamentales en machine learning : la **Grid Search** et la **validation croisée**.

**Grid Search** : Cette méthode m'a permis d'optimiser les hyperparamètres d'un modèle. En testant systématiquement une grille de valeurs prédéfinies, j'ai pu trouver la combinaison qui maximise la performance du modèle.

**Validation croisée** : J'ai utilisé cette technique pour évaluer la robustesse du modèle. En divisant le jeu de données en plusieurs parties (plis), j'ai pu entraîner et tester le modèle sur différentes sous-populations. Cela m'a donné une mesure de performance plus fiable et m'a aidé à éviter le surapprentissage (overfitting).

En combinant ces deux approches, j'ai non seulement pu trouver les meilleurs réglages pour le modèle, mais aussi valider que ces réglages sont performants sur l'ensemble des données, assurant ainsi un modèle plus fiable et généralisable.

### Importations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Données

In [ ]:

df = pd.read_csv("AMES_Final_DF.csv")

In [ ]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


### Création de X et y
**Tâche : Le label ou variable cible que nous essayons de prédire est la colonne SalePrice. Séparez les données en Features X et Label y**.

In [ ]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

### Fractionnement Entraînement Test
**Tâche : Séparer X et y en un ensemble d'entraînement et un ensemble de test. Comme nous utiliserons plus tard une stratégie de recherche par grille (grid search), fixez votre proportion de test à 10 %. **.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=101)

### Mise à l'échelle des données
**Tâche : Les features de l'ensemble de données ont une variété d'échelles et d'unités. Pour une performance de régression optimale, mettez à l'échelle les Features X. Prenez note de ce que vous utilisez pour .fit() et de ce que vous utilisez pour .transform()**.

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Modèle
**Tâche : Nous allons utiliser un modèle Elastic Net. Créez une instance du modèle ElasticNet par défaut avec scikit-learn**.

In [ ]:
model = ElasticNet(max_iter=1000000)

**Tâche : Le modèle Elastic Net a deux paramètres principaux, alpha et le ratio L1 (l1_ratio). Créez un dictionnaire de grille de paramètres des valeurs pour l'Elastic Net.**.

In [ ]:
hyperparameters = {'alpha':[10,100,50],'l1_ratio':[.95,.99,1]}

**Tâche : À l'aide de scikit-learn, créez un objet GridSearchCV et exécutez un Grid Search pour les meilleurs paramètres de votre modèle en fonction de vos données d'entraînement mises à l'échelle. [Au cas où vous seriez curieux de connaître les avertissements que vous pourriez recevoir pour certaines combinaisons de paramètres](https://stackoverflow.com/questions/20681864/lasso-on-sklearn-does-not-converge)**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gs = GridSearchCV(model,hyperparameters,cv=5,verbose=1)

In [ ]:
gs.fit(X_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5, estimator=ElasticNet(max_iter=1000000),
             param_grid={'alpha': [10, 100, 50], 'l1_ratio': [0.95, 0.99, 1]},
             verbose=1)

**Tâche : Afficher la meilleure combinaison de paramètres pour votre modèle**.

In [ ]:
gs.best_estimator_

ElasticNet(alpha=100, l1_ratio=1, max_iter=1000000)

### Évaluation du modèle
Évaluez les performances de votre modèle sur les 10% de données non vues auparavant (ensemble de test mis à l'échelle). Dans le notebook des solutions, nous avons obtenu une MAE de 14195 $\$$ et une RMSE de 20558 $\$$**.

In [ ]:
model = gs.best_estimator_
model.predict(X_test)

In [ ]:
rmse = mean_absolute_error(y_test,model.predict(X_test))
rmse

14195.35490056217